# Modelo número 4


### 1. Carga de librerías


In [ ]:
import pandas as pd
import numpy as np
import gc

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

### 2. Función de preprocesado avanzado

In [ ]:
def preprocesamiento_avanzado_definitivo(df_input):
    df = df_input.copy()

    # Eliminar columnas con un solo valor
    for col in df.columns:
        if df[col].nunique(dropna=False) <= 1:
            df.drop(columns=[col], inplace=True)

    # Imputación de valores nulos para variables categóricas
    for col in df.select_dtypes(include=['object']).columns:
        if col != 'RENDIMIENTO_GLOBAL':
            df[col].fillna(df[col].mode()[0], inplace=True)

    # Conversión automática de columnas binarias 'Sí/No'
    print("Buscando y convirtiendo automáticamente columnas 'Si'/'No'...")
    si_no_map = {'Si': 1, 'No': 0, 'S': 1, 'N': 0}
    for col in df.select_dtypes(include=['object']).columns:
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset(si_no_map.keys()):
            print(f"  -> Columna binaria encontrada: '{col}'. Convirtiendo a 1/0.")
            df[col] = df[col].map(si_no_map)
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int8')

    # Conversión de variables educativas
    edu_map = {'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2, 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4, 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6, 'Postgrado': 7}
    if 'FAMI_EDUCACIONPADRE' in df.columns:
        df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(edu_map)
    if 'FAMI_EDUCACIONMADRE' in df.columns:
        df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(edu_map)

    # Otras transformaciones específicas
    if 'ESTU_VALORMATRICULAUNIVERSIDAD' in df.columns:
        df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map({'No pagó matrícula': 0, 'Menos de 500 mil': 1, 'Entre 500 mil y menos de 1 millón': 2, 'Entre 1 millón y menos de 2.5 millones': 3, 'Entre 2.5 millones y menos de 4 millones': 4, 'Entre 4 millones y menos de 5.5 millones': 5, 'Entre 5.5 millones y menos de 7 millones': 6, 'Más de 7 millones': 7})
    if 'ESTU_HORASSEMANATRABAJA' in df.columns:
        df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map({'0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2, 'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4})
    if 'FAMI_ESTRATOVIVIENDA' in df.columns:
        df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').str.replace('Sin Estrato', '0').astype(np.int8)

    # Crear nuevas variables
    df['INDICE_SOCIOECONOMICO'] = (df['FAMI_TIENEINTERNET'] + df['FAMI_TIENELAVADORA'] + df['FAMI_TIENECOMPUTADOR'] + df['FAMI_TIENEAUTOMOVIL']).astype(np.int8)
    df['PROMEDIO_EDU_PADRES'] = (df['FAMI_EDUCACIONPADRE'] + df['FAMI_EDUCACIONMADRE']).astype(np.float32) / 2.0
    df.drop(['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], axis=1, inplace=True, errors='ignore')

    # Codificación one-hot
    df = pd.get_dummies(df, columns=['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'], dummy_na=False, dtype=np.int8)
    return df



### 3. Carga y procesamiento de datos

In [ ]:
print("Cargando datos...")

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Extraer la columna objetivo (target) del conjunto de entrenamiento
target_col = df_train['RENDIMIENTO_GLOBAL']
test_ids = df_test['ID']

# Eliminar la columna objetivo del conjunto de entrenamiento (solo deben quedar las variables predictoras)
df_train = df_train.drop(columns=['RENDIMIENTO_GLOBAL'])

print("\n--- Procesando datos de entrenamiento ---")

# PREPROCESAMIENTO DE LOS DATOS
train_processed = preprocesamiento_avanzado_definitivo(df_train)
del df_train; gc.collect()

print("\n--- Procesando datos de prueba ---")
test_processed = preprocesamiento_avanzado_definitivo(df_test)
del df_test; gc.collect()

print("\n--- Alineando columnas ---")

# Alinear las columnas entre entrenamiento y prueba (deja solo las que están en ambos conjuntos)
train_final, test_final = train_processed.align(test_processed, join='inner', axis=1, fill_value=0)
del train_processed, test_processed; gc.collect()

# Codificar la variable objetivo (de etiquetas tipo texto a números) usando LabelEncoder
le = LabelEncoder()
y = le.fit_transform(target_col)
del target_col; gc.collect()

# Eliminar la columna 'ID' si existe, ya que no se usa como variable predictora
X = train_final.drop(columns=['ID'], errors='ignore')
X_submission = test_final.drop(columns=['ID'], errors='ignore')
del train_final, test_final; gc.collect()

print("\nDatos listos para modelar.")

Cargando datos...

--- Procesando datos de entrenamiento ---


/tmp/ipython-input-10-1266456060.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


Buscando y convirtiendo automáticamente columnas 'Si'/'No'...
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENELAVADORA'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEAUTOMOVIL'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PRIVADO_LIBERTAD'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PAGOMATRICULAPROPIO'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENECOMPUTADOR'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET.1'. Convirtiendo a 1/0.

--- Procesando datos de prueba ---


/tmp/ipython-input-10-1266456060.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


Buscando y convirtiendo automáticamente columnas 'Si'/'No'...
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENELAVADORA'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEAUTOMOVIL'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PRIVADO_LIBERTAD'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PAGOMATRICULAPROPIO'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENECOMPUTADOR'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET.1'. Convirtiendo a 1/0.

--- Alineando columnas ---

Datos listos para modelar.


### 4. Entrenamiento y validación del Modelo

In [ ]:
# Separar conjunto de validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Entrenando Árbol de Decisión...")
model_dt = DecisionTreeClassifier(max_depth=10, random_state=42)
model_dt.fit(X_train, y_train)

print("Evaluando modelo...")
preds_val = model_dt.predict(X_val)
accuracy = accuracy_score(y_val, preds_val)

print(f"\n-- RESULTADO --")
print(f"Accuracy de Árbol de Decisión en el conjunto de validación: {accuracy:.5f}")

Entrenando Árbol de Decisión...
Evaluando modelo...

-- RESULTADO --
Accuracy de Árbol de Decisión en el conjunto de validación: 0.39244


### 5. Generación del archivo de kaggle

In [ ]:
print("Re-entrenando el modelo de Árbol de Decisión con todos los datos de entrenamiento...")

# Creamos y entrenamos un nuevo modelo con todos los datos
final_model_dt = DecisionTreeClassifier(max_depth=10, random_state=42)
final_model_dt.fit(X, y) # Entrenamos con todo X, no solo con X_train

print("\nGenerando predicciones para el archivo de submission...")
final_predictions_encoded = final_model_dt.predict(X_submission)
final_predictions = le.inverse_transform(final_predictions_encoded)

submission_df = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': final_predictions})
submission_df.to_csv('submission_decision_tree.csv', index=False)

print("\n¡Éxito! El archivo 'submission_decision_tree.csv' ha sido creado.")
print("Primeras 5 filas del archivo:")
print(submission_df.head())

Re-entrenando el modelo de Árbol de Decisión con todos los datos de entrenamiento...

Generando predicciones para el archivo de submission...

¡Éxito! El archivo 'submission_decision_tree.csv' ha sido creado.
Primeras 5 filas del archivo:
       ID RENDIMIENTO_GLOBAL
0  550236         medio-alto
1   98545         medio-bajo
2  499179               bajo
3  782980               bajo
4  785185               bajo
